In [1]:
import sqlite3
import os
import string

In [2]:
#con.close()

In [3]:
os.remove('TransProb_100.db')


In [4]:
con = sqlite3.connect('TransProb_100.db')

In [5]:
crsr = con.cursor()

In [6]:

'''
try:
    dele = "DROP TABLE TransProb"
    crsr.execute(dele)
except:
    print('Error')
'''

'\ntry:\n    dele = "DROP TABLE TransProb"\n    crsr.execute(dele)\nexcept:\n    print(\'Error\')\n'

In [7]:
create_table = """CREATE TABLE TransProb(
ENG_WORD VARCHAR(20) NOT NULL,
DUT_WORD VARCHAR(20) NOT NULL,
PROBABILITY FLOAT(12,10) NOT NULL,
PRIMARY KEY (ENG_WORD,DUT_WORD));
"""

cluster_index="""CREATE INDEX IX_ED_T ON TransProb (
	"ENG_WORD",
	"DUT_WORD"
);"""

DUT_INDEX ="""
        CREATE INDEX IX_DUT ON TransProb (DUT_WORD);
"""

ENG_INDEX ="""
        CREATE INDEX IX_ENG ON TransProb (ENG_WORD);
"""






In [8]:
create_table_c = """CREATE TABLE Count(
ENG_WORD VARCHAR(20) NOT NULL,
DUT_WORD VARCHAR(20) NOT NULL,
PROBABILITY FLOAT(12,10) NOT NULL,
PRIMARY KEY (ENG_WORD,DUT_WORD));
"""

cluster_index_c="""CREATE INDEX IX_ED ON Count (
	"ENG_WORD",
	"DUT_WORD"
);"""

In [9]:
crsr.execute(create_table_c)
crsr.execute(cluster_index_c)

In [10]:
crsr.execute(create_table)

In [11]:
#crsr.execute(DUT_INDEX)
#crsr.execute(ENG_INDEX)

In [12]:
crsr.execute(cluster_index)

In [13]:
import time
import os

In [14]:

UPDATED_ENGLISH_FILE = 'English_Updated.txt'
UPDATED_DUTCH_FILE = 'Dutch_Updated.txt'


ENG_TEST_FILE = 'eng.txt'
DUTCH_TEST_FILE = 'dutch.txt'

In [15]:
e_file = open(UPDATED_ENGLISH_FILE,'r',encoding = 'utf-8')
d_file = open(UPDATED_DUTCH_FILE,'r',encoding = 'utf-8')

In [16]:
dutch = d_file.readlines()
eng = e_file.readlines()

In [17]:
def remove_punc(l,lang):
    
    for i in range(len(l)):
        l[i] = l[i][:-1].lower()
        l[i] = remove_stopwords(l[i],lang)
        l[i] = remove_stuff(l[i])
    return l

def remove_stopwords(l,lang):
    eng_sw=['of', 'the', 'i', 'on', 'and', 'would', 'to', 'you', 'a', 'in', 'that', 'as', 'have', 'for', 'be', 'from', 'it', 'at', 'can', 'an', 'has', 'the', 'it', 'is', 'not', 'with', 'we', 'by', 'this', 'we', 'are', 'more', 'our', 'or', 'also', 'these', 'but', 'must']
    dut_sw=['van', 'de', 'ik', 'aan', 'en', 'zou', 'naar', 'u', 'een', 'in', 'dat', 'als', 'hebben', 'voor', 'worden', 'van', 'het', 'bij', 'kan', 'een', 'heeft', 'de', 'het', 'is', 'niet', 'met', 'wij', 'door', 'deze', 'wij', 'zijn', 'meer', 'onze', 'of', 'ook', 'deze', 'maar', 'moet']
    eng_sw = [i.lower() for i in eng_sw]
    dut_sw = [i.lower() for i in dut_sw]
    
    words = l.split()
    s = []
    
    if lang =='dutch':
        for i in words:
            if i not in dut_sw:
                s.append(i)
    elif lang =='eng':
        for i in words:
            if i not in eng_sw:
                s.append(i)
    return ' '.join(s)

def remove_stuff(l):
    #a = [ ':','.' , '\\' , '/' , ',' , ';' , '(' , ')' , '"', "\'",'1','2','3','4','5','6','7','8','9','0','?']
    for i in string.punctuation + string.digits:
        l = l.replace(i, "")
    
    return l

In [18]:
dutch2 = remove_punc(dutch,'dutch')
eng2 = remove_punc(eng,'eng')

In [19]:
def assign_line_no(doc,dict_lo):
    for i in range(len(doc)):
        t = doc[i].split()
        for m in t:
            dict_lo[m].add(i)
    return dict_lo

In [20]:
from collections import defaultdict

In [21]:
dutch_line_no = defaultdict(set)
eng_line_no = defaultdict(set)

In [22]:
#dutch_line_no
len(dutch_line_no)

0

In [23]:
dutch_line_no = assign_line_no(dutch2, dutch_line_no)
eng_line_no = assign_line_no(eng2, eng_line_no)

In [24]:
def initialize(foreign_no_of_words,foreign_l,english_l):
    count = 0
    init_prob = 1/foreign_no_of_words
    for i in english_l.keys():
        for j in foreign_l.keys():
            c = [i,j,init_prob]
            c1 = [i,j,0]
            try:
                crsr.execute("INSERT INTO TransProb(ENG_WORD, DUT_WORD, PROBABILITY) VALUES(?,?,?)",tuple(c))
                crsr.execute("INSERT INTO Count(ENG_WORD, DUT_WORD, PROBABILITY) VALUES(?,?,?)",tuple(c1))
            except:
                print(c)
        if count>=100:
            con.commit()
            count = 0
        #print(count)
        count+=1
    con.commit()
    return True

In [25]:
b_val = initialize(len(dutch_line_no), dutch_line_no, eng_line_no)

In [26]:
len(dutch_line_no)

7164

In [27]:
t = set()
for k,v in dutch_line_no.items():
    if len((v))==1:
        t.add(list(v)[0])

In [28]:
len(t)

3494